## Hierarchical Classifier using LLMs

A hierarchical classifier using Large Language Models (LLMs) designed to classify text into multiple levels of categories, from broad categories to more granular, specific categories  based on predefined hierarchical structures. 


This sample notebook shows you how to finetune a LLM model for hierarchical classification and infering the results.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. Some hands-on experience using [Amazon SageMaker](https://aws.amazon.com/sagemaker/).
1. To use this algorithm successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to For Seller to update: Arrhythmia Identification from ECG. 

#### Contents:
1. [Subscribe to the algorithm](#1.-Subscribe-to-the-algorithm)
1. [Prepare dataset](#2.-Prepare-dataset)
	1. [Dataset format expected by the algorithm](#A.-Dataset-format-expected-by-the-algorithm)
	1. [Configure dataset](#B.-Configure-dataset)
	1. [Upload datasets to Amazon S3](#C.-Upload-datasets-to-Amazon-S3)
1. [Train a machine learning model](#3:-Train-a-machine-learning-model)
	1. [Set up environment](#3.1-Set-up-environment)
	1. [Train a model](#3.2-Train-a-model)
1. [Deploy model and verify results](#4:-Deploy-model-and-verify-results)
    1. [Deploy trained model](#A.-Deploy-trained-model)
    1. [Create input payload](#B.-Create-input-payload)
    1. [Perform real-time inference](#C.-Perform-real-time-inference)
    1. [Visualize output](#D.-Visualize-output)
    1. [Calculate relevant metrics](#E.-Calculate-relevant-metrics)
    1. [Delete the endpoint](#F.-Delete-the-endpoint)
1. [Perform Batch inference](#6.-Perform-Batch-inference)
1. [Clean-up](#7.-Clean-up)
	1. [Delete the model](#A.-Delete-the-model)
	1. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))


#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

# 1. Subscribe to the algorithm

To subscribe to the algorithm:
1. Open the algorithm listing page Hierarchical classifier.
1. On the AWS Marketplace listing,  click on **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you agree with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn**. This is the algorithm ARN that you need to specify while training a custom ML model. Copy the ARN corresponding to your region and specify the same in the following cell.

In [82]:
algo_name = 'hierarchical-classifier-v8-1'

### 2. Prepare dataset

In [1]:
import base64
import json 
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
from urllib.parse import urlparse
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit
import urllib.request
import numpy as np
import boto3
import sagemaker

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


#### A. Dataset format expected by the algorithm

Usage Instructions:
- For model finetuning, the data can be uploaded in a folder as train.csv file. 
    - Adhere to the naming convention mentioned below for finetuning the model.
        - The textual description must have the column name DESCRIPTION
        - The categories must have the column names like CATEGORY_1, CATEGORY_2, CATEGORY_3, CATEGORY_4
    - The total number of labels in all the category columns together must not exceed 256   
- For inference all the textual description must be uploaded in a dictionary format in a json file like below:     
        - {"text":<description_1>,<description_2>,<description_3>}
- Correct naming convention should be followed for files uploaded. 
- Inference engine provides predictions for each textual description.

#### B. Configure dataset

In [84]:
training_dataset="train.csv"
test_dataset = "inference.json"
test_dataset_batch = "inference_batch.json"

#### C. Upload datasets to Amazon S3

In [ ]:
sagemaker_session = sage.Session()
bucket=sagemaker_session.default_bucket()
bucket

In [ ]:
project = 'hierarchical-classifier'

# Specify the file to upload
local_file_path = 'train_half_data.csv'  # Replace with your local file path
test_input_prefix = f"{project}/input/test"
training_input_prefix = f"{project}/input/train"     # Specify the S3 object key (path in S3)

training_input=sagemaker_session.upload_data(training_dataset, bucket=bucket, key_prefix=training_input_prefix)
print("Training input uploaded to " + training_input)

test_input=sagemaker_session.upload_data(test_dataset, bucket=bucket, key_prefix=test_input_prefix)
test_input_batch=sagemaker_session.upload_data(test_dataset_batch, bucket=bucket, key_prefix=test_input_prefix)

print("Test input uploaded to " + test_input)
print("Test batch input uploaded to " + test_input_batch)



## 3: Train a machine learning model

Now that dataset is available in an accessible Amazon S3 bucket, we are ready to train a machine learning model. 

### 3.1 Set up environment

In [87]:
role = get_execution_role()

In [ ]:
output_location = f's3://{bucket}/{project}/'.format(bucket, 'output')
output_location

### 3.2 Train a model

In [89]:
training_instance_type="ml.g5.4xlarge"

In [ ]:
#Create an estimator object for running a training job
hyperparameters = {
    'epochs': 20
}

estimator = sage.algorithm.AlgorithmEstimator(
    algorithm_arn=algo_name,
    base_job_name=project,
    role=role,
    hyperparameters=hyperparameters,
    train_instance_count=1,
    train_instance_type=training_instance_type,
    input_mode="File",
    max_run=86400,
    output_path=output_location,
    sagemaker_session=sagemaker_session,
    instance_count=1,
    instance_type=training_instance_type
)

#Run the training job.
estimator.fit({"train": training_input})

INFO:sagemaker:Creating training-job with name: hierarchical-classifier-2024-12-05-16-38-39-079


2024-12-05 16:38:40 Starting - Starting the training job...
2024-12-05 16:38:53 Starting - Preparing the instances for training...
2024-12-05 16:39:28 Downloading - Downloading the training image...............
2024-12-05 16:42:00 Training - Training image download completed. Training in progress......Num training epochs: 1
input_path: /opt/ml/input/data
training_path: /opt/ml/input/data/train
base_model path: /opt/ml/code/base_model
#015Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]#015Loading checkpoint shards:  50%|█████     | 1/2 [00:00<00:00,  1.13it/s]#015Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.51it/s]#015Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.44it/s]
Tokenizer loaded!!
training_path: /opt/ml/input/data/train
Training data is loaded
Total category columns: ['CATEGORY_1', 'CATEGORY_2']
Total unique categories across all columns: 42
/opt/ml/code/train:99: SettingWithCopyWarning: 
A value is trying to be set on a c

### 4: Deploy model and verify results

Now you can deploy the model for performing real-time inference.

In [93]:
content_type='application/json' #input

real_time_inference_instance_type='ml.p3.2xlarge'
batch_transform_inference_instance_type='ml.p3.2xlarge'

#### A. Deploy trained model

In [94]:
predictor = estimator.deploy(1, real_time_inference_instance_type)

INFO:sagemaker:Creating model package with name: hierarchical-classifier-2024-12-05-17-00-08-688


.........

INFO:sagemaker:Creating model with name: hierarchical-classifier-2024-12-05-17-00-08-688


INFO:sagemaker:Creating endpoint-config with name hierarchical-classifier-2024-12-05-17-00-08-688
INFO:sagemaker:Creating endpoint with name hierarchical-classifier-2024-12-05-17-00-08-688


-------------!

Once endpoint is created, you can perform real-time inference.

#### B. Create input payload

In [95]:
file_name = test_dataset
output_file_name = "inference_out.json"

#### C.Perform real-time inference

In [96]:
from sagemaker.predictor import Predictor

endpoint_name = predictor.endpoint
predictor = Predictor(endpoint_name)


See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [97]:
import boto3
import json

# Read the JSON input data from the file
with open(test_dataset, 'r') as file:
    input_data = json.load(file)  # This loads the JSON into a Python dictionary

# Convert the Python dictionary to a JSON string
json_input_data = json.dumps(input_data)

# Create a SageMaker Runtime client
client = boto3.client('sagemaker-runtime', region_name='us-east-2')  # Update your region

# Define the endpoint name
endpoint_name = predictor.endpoint

# Perform inference by invoking the endpoint
response = client.invoke_endpoint(
    EndpointName=endpoint_name,  # The name of the deployed endpoint
    Body=json_input_data,        # Input data as a JSON string
    ContentType='application/json',  # Indicating that the content type is JSON
    Accept='application/json'    # Expecting JSON format response
)

# Get the response
result = json.loads(response['Body'].read().decode())  # Parse the JSON response
print(result)


See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


{'prediction': ['Service Request,GL Account – Reopen']}


### D. Visualize output

In [101]:
print(result)

{'prediction': ['Service Request,GL Account – Reopen']}


#### F. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. you can terminate the same to avoid being charged.

In [102]:
predictor.delete_endpoint(delete_endpoint_config=True)

INFO:sagemaker:Deleting endpoint configuration with name: hierarchical-classifier-2024-12-05-17-00-08-688
INFO:sagemaker:Deleting endpoint with name: hierarchical-classifier-2024-12-05-17-00-08-688


Since this is an experiment, you do not need to run a hyperparameter tuning job. However, if you would like to see how to tune a model trained using a third-party algorithm with Amazon SageMaker's hyperparameter tuning functionality, you can run the optional tuning step.

### 5. Perform Batch inference

In this section, you will perform batch inference using multiple input payloads together.

In [ ]:
#Run the batch-transform job
transformer = estimator.transformer(1, batch_transform_inference_instance_type)
transformer.transform(test_input_batch, content_type=content_type)
transformer.wait()

INFO:sagemaker:Creating model package with name: hierarchical-classifier-v8-1-2024-12-05-17-12-08-660


.........

INFO:sagemaker:Creating model with name: hierarchical-classifier-v8-1-2024-12-05-2024-12-05-17-12-54-184


INFO:sagemaker:Creating transform job with name: hierarchical-classifier-2024-12-05-17-12-54-935


..................................................
2024-12-05T17:21:28.935:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
FULL PATH: /opt/ml/model/final_model_checkpoint
FULL PATH: /opt/ml/model/final_model_checkpoint
#015Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]#015Loading checkpoint shards:  50%|█████     | 1/2 [00:01<00:01,  1.14s/it]#015Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.18it/s]#015Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.12it/s]
loaded model from: /opt/ml/code/base_model
loaded adapter from: final_model_checkpoint
cuda:0
before starting app
 * Serving Flask app 'serve'
 * Debug mode: off
#033#033WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.#033
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://169.254.255.131:8080
#033Press CTRL+C to quit#033
work

In [ ]:
#output is available on following path
transformer.output_path

### 7. Clean-up

#### A. Delete the model

In [105]:
estimator.delete_endpoint()

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the algorithm, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

